# Installation: 
To train an object detection model on a custom dataset, there are 3 prerequisites:
* Install TensorFlow
* Install TensorFlow Object Detection API
* Install labelImg (for dataset annotation)

### Installing TensorFlow

Installing Anaconda is a useful tool for creating virtual environments and managing packages. However, it is optional. 
It's recommended to use a virtual environment. We can simply use pip to install tensorflow within the virtual environment:

`!pip install tensorflow`

Also will need to install the setup tools package for python:

`apt-get install install -y python-setuptools`

### Installing TensorFlow Object Detection API

#### Install Dependencies

Installing and integrating the TensorFlow Object Detection API is slightly more involved than simply using pip to install a package, as the object detection API does not come packaged with TensorFlow. 
There are several prerequisites that must be installed first:
* pillow
* lxml
* jupyter
* matplotlib
* opencv
* cython

These packages can be installed using anaconda by running:

`conda install pillow, lxml, jupyter, matplotlib, opencv, cython`

Alternatively (recommended), one can use pip to install the dependencies. We also need to install the protobuf compiler:

`apt-get install protobuf-compiler python-pil python-lxml python-tk -y`

Then we can run the pip install commands:

`pip install pillow lxml jupyter matplotlib opencv-python cython contextlib2`

#### Download TensorFlow Models
During the integration and compilation process for the Object Detection API, it'll be useful to keep the directories organized in a hierarchial fashion. You can create a new folder named TensorFlow, then use git to clone the TensorFlow Models repo 
(https://github.com/tensorflow/models) inside the TensorFlow folder. The command should be:

`git clone https://github.com/tensorflow/models.git`

Your TensorFlow folder should be organized in this format now:

`TensorFlow
    |-- models
        |-- official
        |-- research
        |-- samples
        |-- tutorials`

#### Compile Object Detection API
We need to compile to proto files within the framework before we can run the API.

Navigate to the research directory, then run these commands:

`
wget -O protobuf.zip https://github.com/google/protobuf/releases/download/v3.0.0/protoc-3.0.0-linux-x86_64.zip
unzip -o protobuf.zip
./bin/protoc object_detection/protos/*.proto --python_out=.
bash object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
{sys.executable} -m setup.py sdist
(cd slim && {sys.executable} -m setup.py sdist)
cd ../..
`

Next, we need to add the object detection API to the local environment by running these commands via python (Or just use cp on terminal):

`
os.mkdir("./packages")
shutil.copy("./models/research/dist/object_detection-0.1.tar.gz","./packages/object_detection-0.1.tar.gz")
shutil.copy("./models/research/slim/dist/slim-0.1.tar.gz","./packages/slim-0.1.tar.gz")
shutil.copy("/tmp/pycocotools/pycocotools-2.0.tar.gz","./packages/pycocotools-2.0.tar.gz")
`

Then, go back to terminal and install these packages:

`
!{sys.executable} -m pip install ./packages/object_detection-0.1.tar.gz
!{sys.executable} -m pip install ./packages/slim-0.1.tar.gz
!{sys.executable} -m pip install ./packages/pycocotools-2.0.tar.gz
` 

And lastly, from inside the research folder, add the TensorFlow object detection API packages to your PYTHONPATH just to confirm. <PATH_TO_TF> should replace the absolute path to your TensorFlow folder:

`
export PYTHONPATH=$PYTHONPATH:<PATH_TO_TF>/TensorFlow/models/research:<PATH_TO_TF>/TensorFlow/models/research/slim
`


### Installing LabelImg

LabelImg can be downloaded using git, but has a different dependency than TensorFlow, so will require a new virtual environment to run labelImg:

`conda create -n labelImg py`

Once the virtual environment is created, activate it:

`activate labelImg`

Create a new directory inside the TensorFlow folder and name it addons. Once inside the new folder, you can clone the labelImg repository using git:

`git clone https://github.com/tzutalin/labelImg.git`

Inside the virtual environment, you'll need to install the dependencies and compile the labelImg package:

`conda install pyqt=4
conda install lxml
pyrcc4 -py3 -o resources.py resources.qrc`

After these installation steps for labelImg, you can test it out by changing directory to the labelImg folder and running it:

`python labelImg.py`

Now we're ready to start training!

# Custom Object Detector Training

Inside the TensorFlow folder, create a new folder named workspace, where we can store all working files. Inside, we can create the directory for our actual project, named SignLanguageAlphabetDetection. The repository should look like this:

`TensorFlow
    |-- models
        |-- official
        |-- research
        |-- samples
        |-- tutorials
    |-- addons
        |-- labelImg
    |-- workspace
        |-- SignLanguageAlphabetDetection
`

Within the SignLanguageAlphabetDetection folder, we can further organize our files in such a manner:

`SignLanguageAlphabetDetection
    |-- annotations
    |-- images
        |-- train
            |-- JPG
            |-- XML
        |-- test
            |-- JPG
            |-- XML
    |-- models
`

### Create Label Map

To identify object classes, a label map is required to map each labels to integer values.

In [ ]:
%%writefile annotations/label_map.pbtxt

item {
    id: 1
    name: 'hand'
}

### Preprocessing: Create TensorFlow Records

To train the model, we need to convert the annotated XML files into a TensorFlow record.

In [ ]:
%matplotlib inline

import sys
import shutil
import os
import io
import glob
from PIL import Image
from lxml import etree
import random
import contextlib2

import tensorflow as tf
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util
from object_detection.dataset_tools import tf_record_creation_util

import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

In [ ]:
data_dir = './dataset'
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'

label_map_path = 'annotations/label_map.pbtxt'
output_dir = 'tfrecords/'
os.mkdir(output_dir)

In [ ]:
label_map_dict = label_map_util.get_label_map_dict(label_map_path)
class_name = 'hand'

In [ ]:
# Set up filepaths for tensorflow record files to be created from dataset
train_output_path = os.path.join(output_dir, 'train.record')
test_output_path = os.path.join(output_dir, 'test.record')

In [ ]:
# Check how many images in dataset
training_images = glob.glob(train_dir + "/JPG/*.jpg")
test_images = glob.glob(test_dir + "/JPG/*.jpg")
print("There are {} training images.".format(len(training_images)))
print("There are {} test images.".format(len(test_images)))

In [ ]:
def dict_to_tf_example(data, label_map_dict, img_path):
    """
    Converts dictionary of a single image and its xml annotation into a Tensorflow compatible example.
    
    Args:
        data (dictionary): contains xml annotation data (image dims, bounding box coordinates and labels)
        label_map_dict (dictionary): contains mapping of out_of_stock class to its id
        img_path (str): filepath for image to encode
    Returns:
        TensorFlow Example containing encoded data from xml annotation and image data
    """
    with tf.Graph().as_default():
        image_contents = tf.read_file(img_path)
        image = tf.image.decode_jpeg(image_contents, channels=3)
        init_op = tf.initialize_all_tables()
        with tf.Session() as sess:
            sess.run(init_op)
            try:
                tmp = sess.run(image)
            except:
                print(img_path)
    with tf.io.gfile.GFile(img_path, 'rb') as file:
        encoded_jpeg = file.read()
    encoded_jpeg_io = io.BytesIO(encoded_jpeg)
    image = Image.open(encoded_jpeg_io)
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    width = int(data['size']['width'])
    height = int(data['size']['height'])
    xmins, xmaxs, ymins, ymaxs = [], [], [], []
    classes, classes_text = [], []
    
    if 'object' in data:
        for obj in data['object']:
            xmins.append(float(obj['bndbox']['xmin']) / width)
            xmaxs.append(float(obj['bndbox']['xmax']) / width)
            ymins.append(float(obj['bndbox']['ymin']) / height)
            ymaxs.append(float(obj['bndbox']['ymax']) / height)
            classes_text.append(class_name.encode('utf8'))
            classes.append(label_map_dict[class_name])
    feature_dict = {
        'image/width': dataset_util.int64_feature(width),
        'image/height': dataset_util.int64_feature(height),
        'image/filename': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_jpeg),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes)
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example            

In [ ]:
def create_tf_record(output_filename, num_shards, label_map_dict, images):
    """
    Writes a Tensorflow Record file from the xml annotations of provided images and splits up records into shards.
    
    Args:
        output_filename (str): filepath of TensorFlow record to write
        num_shards (int): number of shards to divide records into
        label_map_dict (dictionary): contains mapping for out_of_stock identifier and label
        images ([Str]): list of strings of images' filepaths
    Returns:
        None
    """
    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filename, num_shards)
        for idx, image in enumerate(images):
            if idx % 50 == 0:
                print(f"On image {idx} of {len(images)}.")
            split_path = image.split("JPG/")
            xml_path = os.path.join(split_path[0] + "XML/" + split_path[1][:-4] + ".xml")
            if not os.path.exists(xml_path):
                print(f"Could not find {xml_path}, ignoring image.")
                continue
            with tf.io.gfile.GFile(xml_path, 'r') as file:
                xml_str = file.read()
            xml = etree.fromstring(xml_str)
            data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
            try:
                tf_example = dict_to_tf_example(data, label_map_dict, image)
                if tf_example:
                    shard_idx = idx % num_shards
                    output_tfrecords[shard_idx].write(tf_example.SerializeToString())
            except ValueError:
                print(f"Invalid example {xml_path}, ignoring.")

In [ ]:
# Create tensorflow records.
create_tf_record(train_output_path, 2, label_map_dict, training_images)
create_tf_record(test_output_path, 2, label_map_dict, test_images)

### Training Model

Before running the training job, we need a model to train on.
Check out https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md and select which model to train. Once you've selected a model, copy the link address and download it on terminal via the wget command (Download link from the list):

`wget <TAR FILE LINK> (ex. http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz)`

Once downloaded and placed in the models folder within your workspace, unzip the tar file via:

`tar -xzvf <TAR FILE NAME>`

Next, you need to configure the training pipeline job. The unzipped model folder should have a pipeline.config file, in which you need to edit some fields to match the required number of object classes, path to training/test tensorflow records, and path to label map. Look at https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html to see details regarding the pipeline file.

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/running_locally.md